In [2]:
#Dependencies 
import numpy as np
import pandas as pd
import requests
import time
import json
from us import states
from scipy.stats import linregress
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'us'

In [ ]:
url = "https://data.cdc.gov/resource/vbim-akqf.json"

In [ ]:
print(requests.get(url))

In [ ]:
print(requests.get(url).json())

In [ ]:
url = "https://data.cdc.gov/resource/vbim-akqf.json"
response = requests.get(url)
response_json = response.json()
print(json.dumps(response_json, indent=4, sort_keys=True))

In [3]:
cases_df =  pd.DataFrame({
    "State": ["Georgia", "California", "Illinois", "Texas", "New York"],
    "Cases": [175052, 466550, 174973, 394265, 225850]})
cases_df

,State,Cases
0,Georgia,175052
1,California,466550
2,Illinois,174973
3,Texas,394265
4,New York,225850


In [48]:
########### Dave's code below for 2017-18 flu ################
# import and clean data from CDC FluView website csv download

In [49]:
# read data file
raw_state_flu_2017 = pd.read_csv("../project_data/State_ILINet.csv")

In [50]:
raw_state_flu_2017

,REGION TYPE,REGION,YEAR,WEEK,% WEIGHTED ILI,%UNWEIGHTED ILI,AGE 0-4,AGE 25-49,AGE 25-64,AGE 5-24,AGE 50-64,AGE 65,ILITOTAL,NUM. OF PROVIDERS,TOTAL PATIENTS
0,States,Alabama,2017,40,X,1.75721,X,X,X,X,X,X,698,85,39722
1,States,Alaska,2017,40,X,2.72675,X,X,X,X,X,X,141,17,5171
2,States,Arizona,2017,40,X,2.3554,X,X,X,X,X,X,406,42,17237
3,States,Arkansas,2017,40,X,0.812294,X,X,X,X,X,X,37,17,4555
4,States,California,2017,40,X,1.75421,X,X,X,X,X,X,852,102,48569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2647,States,Virginia,2018,39,X,1.33406,X,X,X,X,X,X,1194,127,89501
2648,States,Washington,2018,39,X,0.22417,X,X,X,X,X,X,11,21,4907
2649,States,West Virginia,2018,39,X,0.71708,X,X,X,X,X,X,44,19,6136
2650,States,Wisconsin,2018,39,X,1.00353,X,X,X,X,X,X,37,20,3687


In [51]:
# collect a list of all columns within the DataFrame
raw_state_flu_2017.columns

Index(['REGION TYPE', 'REGION', 'YEAR', 'WEEK', '% WEIGHTED ILI',
       '%UNWEIGHTED ILI', 'AGE 0-4', 'AGE 25-49', 'AGE 25-64', 'AGE 5-24',
       'AGE 50-64', 'AGE 65', 'ILITOTAL', 'NUM. OF PROVIDERS',
       'TOTAL PATIENTS'],
      dtype='object')

In [52]:
# purge unneeded columns
temp_state_flu_2017 = raw_state_flu_2017[['REGION', 'YEAR', 'WEEK', 'ILITOTAL']]
temp_state_flu_2017.head()

,REGION,YEAR,WEEK,ILITOTAL
0,Alabama,2017,40,698
1,Alaska,2017,40,141
2,Arizona,2017,40,406
3,Arkansas,2017,40,37
4,California,2017,40,852


In [53]:
# identify incomplete rows
temp_state_flu_2017.count()

REGION      2652
YEAR        2652
WEEK        2652
ILITOTAL    2652
dtype: int64

In [60]:
#check data types
temp_state_flu_2017.dtypes

REGION      object
YEAR         int64
WEEK         int64
ILITOTAL    object
dtype: object

In [61]:
# drop rows with "X" in ILITOTAL column

# get names of indexes for which column ILITOTAL has value "X"
indexNames = temp_state_flu_2017[temp_state_flu_2017['ILITOTAL'] == 'X' ].index

# Delete these row indexes from dataFrame
temp_state_flu_2017.drop(indexNames , inplace=True)

In [62]:
#check data types
temp_state_flu_2017.dtypes

REGION      object
YEAR         int64
WEEK         int64
ILITOTAL    object
dtype: object

In [63]:
# use pd.to_numeric() method to convert the datatype of the ILITOTAL column
temp_state_flu_2017['ILITOTAL'] = pd.to_numeric(temp_state_flu_2017['ILITOTAL'])

In [64]:
temp_state_flu_2017['ILITOTAL'].dtype

dtype('int64')

In [65]:
# ******* NOT WORKING ******* NOT WORKING ******* NOT WORKING *********
# rename some columns     
clean_state_flu_2017 = temp_state_flu_2017.rename(columns={"State":"Year", "Week":"ILI Cases"})
clean_state_flu_2017

,REGION,YEAR,WEEK,ILITOTAL
0,Alabama,2017,40,698
1,Alaska,2017,40,141
2,Arizona,2017,40,406
3,Arkansas,2017,40,37
4,California,2017,40,852
...,...,...,...,...
2647,Virginia,2018,39,1194
2648,Washington,2018,39,11
2649,West Virginia,2018,39,44
2650,Wisconsin,2018,39,37


In [74]:
# use GroupBy to separate the data into fields according to "State" values
#state_cases = clean_state_flu_2017.groupby(['ILITOTAL'])
#state_cases.sum().head()

# The object returned is a "GroupBy" object and cannot be viewed normally...
#print(state_cases)

# In order to be visualized, a data function must be used...
#state_cases.count().head(10)